## Initialization

In [1]:
import arrow
import datetime
import numpy as np
import os
import pandas as pd
import requests
import sys

In [2]:
# with open('keenan_token.csv') as f:
#     secret_token = f.read().strip()

In [3]:
secret_token = 'YOUR SECRET TOKEN HERE'

In [4]:
# DO NOT USE THIS! USE kensho_external_graph_client

class KenshoGraphClient(object):
    """Basic implementation of the Kensho Graph API client"""

    def __init__(self, host, api_key, converter_func, retries=1):
        """Initialize the client"""
        self._host = host
        self._api_key = api_key
        self._retries = retries
        self._converter_func = converter_func

    def list_entity_classes(self):
        """List all available entity classes"""
        return self._get_json_or_raise('list_entity_classes')

    def list_timeline_types(self):
        """List all available entity classes"""
        return self._get_json_or_raise('list_timeline_types')

    def get_class_relationships(self, class_name):
        """Return all potential relationships of an entity of this class"""
        return self._get_json_or_raise(
            'get_class_relationships', class_name=class_name)

    def search_entities(self, class_name, search_string):
        """Search entities of the class_name that match search_string"""
        return self._get_json_or_raise(
            'search_entities', class_name=class_name, search_string=search_string)

    def list_entities_of_class(self, class_name):
        """List all entities of class"""
        return self._get_json_or_raise(
            'list_entities_of_class', class_name=class_name)

    def get_entity(self, entity_id):
        """Get entity by id"""
        return self._get_json_or_raise('get_entity', entity_id=entity_id)

    def get_related_entities(self, entity_id, relationship):
        """Get all entities related by one identified by entity_id by 'relationship'"""
        return self._get_json_or_raise(
            'get_related_entities', entity_id=entity_id, relationship=relationship)

    def get_timeline(self, timeline_id, start_date=None, end_date=None):
        """Get all events in a timeline. Optionally bound by start and end dates"""
        return self._get_json_or_raise(
            'get_timeline', timeline_id=timeline_id, start_date=start_date, end_date=end_date)

    def get_calendar(self, start_date, end_date):
        """Get all events happening in the interval [start_date, end_date)"""
        return self._get_json_or_raise('get_calendar', start_date=start_date, end_date=end_date)

    def get_ongoing_episodes(self, start_date, end_date):
        """Get all episodes ongoing in the interval [start_date, end_date)"""
        return self._get_json_or_raise(
            'get_ongoing_episodes', start_date=start_date, end_date=end_date)

    def translate_asset_id(self, id_type, asset_id):
        """Given an identification string for an asset return all known identifiers"""
        return self._get_json_or_raise('translate_asset_id', id_type=id_type, asset_id=asset_id)

    def _get_json_or_raise(self, function, **kwargs):
        """Get json from a given url. Uses get_with_retries underneath"""
        session = requests.Session()
        session.mount('https://', requests.adapters.HTTPAdapter(max_retries=self._retries))
        full_url = os.path.join(self._host, function)
        headers = {
            'Authorization': 'Token {}'.format(self._api_key),
            'Content-Type': 'application/json'
        }
        response = session.get(full_url, headers=headers, params=kwargs)
        response.raise_for_status()
        result_json = response.json()
        return self._converter_func(result_json)


def api_result_to_frame(api_result):
    """Convert API result into dataframe"""
    ordered_columns = [c['key'] for c in api_result['metadata']]
    # Create the dataframe
    df = pd.DataFrame(api_result['data'])
    existence_filtered_columns = [c for c in ordered_columns if c in df.columns]
    reordered_frame = df[existence_filtered_columns]

    for column in api_result['metadata']:
        if column['key'] not in reordered_frame.columns:
            continue
        if column['unit'] == 'DateTime' or column['unit'] == 'Date':
            reordered_frame[column['key']] = pd.to_datetime(reordered_frame[column['key']])
    return reordered_frame
    
def get_pandas_graph_client(host, api_key, retries=1):
    """Get a client that returns raw json"""
    return KenshoGraphClient(host, api_key, api_result_to_frame, retries=retries)


In [5]:
graph_client = get_pandas_graph_client('https://www.kensho.com/external/v1/', secret_token)

In [6]:
def ignore_timeline_types(df, timeline_types_to_ignore):
    if isinstance(timeline_types_to_ignore, str):
        raise ValueError('timeline_types_to_ignore may not be a string: {}'.format(timeline_types_to_ignore))
        
    indices = None
    for timeline_type in timeline_types_to_ignore:
        new_indices = df['timeline_type'] != timeline_type
        if indices is None:
            indices = new_indices
        else:
            indices &= new_indices
        
    return df[indices]

## List Functions

#### What kind of timelines are there in the Kensho Knowledge Graph?

In [7]:
graph_client.list_timeline_types()

,timeline_type
0,Academia
1,Activism
2,Administrative
3,Advisory
4,All Time High
5,Analyst Meetings
6,Analyst/Investor Day
7,Announcement of Interim Management Statement
8,Annual General Meeting
9,Appointments


#### What kind of entities are there in the Kensho Knowledge Graph? 

In [8]:
graph_client.list_entity_classes()

,class_name,can_list_entities
0,Sector,True
1,Concept,True
2,TimelineType,True
3,Timeline,False
4,Region,True
5,Currency,True
6,BankMeeting,False
7,Equity,False
8,CentralBank,True


#### Nice, so you have nine broad entity classes. How do I see all of the entities that comprise a single entity class? 

##### Example One: Currencies

In [9]:
graph_client.list_entities_of_class('Region').sort_values('entity_name').head(10)

,entity_name,entity_type,entity_id,entity_aliases,region_type,parent_name,parent_id
226,Adriatic Sea,Region,cda98276-ed5b-40cf-94e3-fb19bbcec488,[],Region,World,79bd87da-5cb9-45b7-8d1b-bdd3c21542ff
295,Aegean Sea,Region,d360b359-2d86-41a3-9606-009c61883c2e,[],Region,World,79bd87da-5cb9-45b7-8d1b-bdd3c21542ff
112,Afghanistan,Region,2f902415-6f14-4180-8e1b-e240b13b9736,"[Afghan, Islamic Republic of Afghanistan]",Country,Southern Asia,3df93e10-dfe1-4b8b-a8d4-d11e19b0c952
141,Africa,Region,6f12e428-1ea6-4b58-97b0-a8c9a5915d3f,[African],Region,"Europe, the Middle East and Africa",2a6f5acf-45a1-4189-bef4-2ca20b1347e9
246,Albania,Region,89891545-4da9-41dc-bee5-fe7117fcc4f3,"[Albanian, Republika e Shqipërisë, Shqipërisë,...",Country,Southern Europe,805534df-3823-4b83-9f36-bff2d08caf82
313,Algeria,Region,f06d884a-c362-4d93-8e63-dd3ccec06944,"[People’s Democratic Republic of Algeria, PDRA...",Country,Northern Africa,63716670-5806-45cf-9f31-e56e4725a794
45,American Samoa,Region,664e47af-cf2f-458d-8d9d-81fe46a7c4cb,[],Country,Polynesia,03755922-6cbb-4747-8b5e-0b7d023bed91
209,Americas,Region,ad7821cf-ed52-40ee-b991-b9147779ccef,[],Region,World,79bd87da-5cb9-45b7-8d1b-bdd3c21542ff
240,Andean Common Market,Region,0f4948b4-cb6e-4d43-8230-d514819d12df,[ANCOM],EconomicRegion,NaN,NaN
115,Andorra,Region,d331cc3e-a488-4e1b-a63d-5831b319fd0c,"[Principality of the Valleys of Andorra, Andor...",Country,Southern Europe,805534df-3823-4b83-9f36-bff2d08caf82


In [10]:
graph_client.list_entities_of_class('Concept').sort_values('entity_name')

,entity_name,entity_type,entity_id,entity_aliases
14,3D Printing,Concept,262737ff-18bc-492c-b460-2d96bd79f6c3,"[additive manufacturing, am]"
11,4K TV,Concept,83cad2d0-fed4-4d58-8922-5302dd74307a,"[4k resolution, 4k ultra high definition, 4k uhd]"
27,5G,Concept,dc93dd39-1b44-4278-8952-613b779b73fa,[]
1,Alcohol,Concept,78218550-cdb3-494d-b5de-c257e02a82b0,[]
10,Artificial Intelligence,Concept,7f4a32d8-9d6f-4087-8c42-3bed8e57a171,"[ai, machine intelligence, mi]"
17,Augmented Reality,Concept,4bbb8315-eabd-4a65-a0cc-4b1de43c58db,[ar]
6,Autonomous Vehicles,Concept,257edbf7-6185-4584-a5c4-91ac0b7bcbb3,"[driverless cars, self-driving cars, robot car..."
2,Cryptocurrency,Concept,e7ffc49f-3f96-4925-bf14-63d2438b1a8e,"[blockchain, crypto currency]"
8,Cybersecurity,Concept,5fa35db7-3437-4853-9802-4a6490b7f0ce,"[computer security, it security, cybersecurity]"
12,Data Storage,Concept,b17035bf-de7c-4b6b-941f-1badb581b985,"[data management, cloud storage]"


##### Example Two: Regions - How many countries / regions / economic regions are in the Kensho Knowledge Graph?

In [11]:
graph_client.list_entities_of_class('Region')['region_type'].value_counts()

Country           249
Region             65
EconomicRegion     19
InformalRegion      3
Name: region_type, dtype: int64

## Search Functions

#### Timelines

##### Example One: What Federal Open Markets Committee (FOMC) timelines are in the Kensho Knowledge Graph? 

In [12]:
graph_client.search_entities('Timeline', 'fomc')

,entity_name,entity_type,entity_id,entity_aliases,timeline_type,related_entities
0,Federal Open Markets Committee Meeting Minutes,Timeline,5d28d014-370c-4415-b6d3-3f1f54c0a363,[],Central Bank Meetings,[{'entity_id': '7754b877-b037-423b-8ab5-d0aba6...
1,Federal Open Markets Committee Meeting Statements,Timeline,bd2585f6-f5b9-4e0f-81a3-e1a77c7df4a2,[],Central Bank Meetings,[{'entity_id': '7754b877-b037-423b-8ab5-d0aba6...
2,Federal Open Markets Committee Meeting Start,Timeline,eb5e5f23-3080-49a1-b5d8-df6f44c70bcd,[],Central Bank Meetings,[{'entity_id': '7754b877-b037-423b-8ab5-d0aba6...
3,Federal Open Markets Committee Meeting End,Timeline,0b2fa0d6-02e1-43db-a5c1-bfba327ad017,[],Central Bank Meetings,[{'entity_id': '7754b877-b037-423b-8ab5-d0aba6...
4,"""No Hike"" FOMC Meetings",Timeline,ecccc837-43d3-4a09-9ae8-32726c1fd20f,[],Central Bank Meetings,[{'entity_id': '42aafef6-053e-4f03-af83-9c9320...
5,Federal Open Markets Committee Chair Speeches,Timeline,6a6e8724-97b0-4fc5-98c5-331727dead9e,[],Communications,[{'entity_id': '650505b8-ead1-45ee-93ba-7369a4...
6,Federal Open Markets Committee Leaves Interest...,Timeline,19bb1e6a-130a-4559-839d-ee3747f44238,[],Rate Actions,[{'entity_id': '7754b877-b037-423b-8ab5-d0aba6...
7,Hawkish FOMC Meetings since 1990,Timeline,989127fc-4d5d-4b21-bbaa-5bd0d2cbae3e,[],Central Bank Meetings,[{'entity_id': '7754b877-b037-423b-8ab5-d0aba6...
8,Dovish FOMC Meetings since 1990,Timeline,ec15d342-625e-4007-bac9-c192af95fcb5,[],Central Bank Meetings,[{'entity_id': '7754b877-b037-423b-8ab5-d0aba6...
9,U.S. Federal Reserve Blackout Period Start Dates,Timeline,d0261181-1287-41fe-9fff-a55626ebdea8,[],Central Bank Speeches,[{'entity_id': '7754b877-b037-423b-8ab5-d0aba6...


In [13]:
#graph_client.search_entities('Timeline', 'labor')

In [14]:
#graph_client.search_entities('Timeline', 'nfl')

##### Example Two: I just saw a video of Jaysum Tatum breaking ankles at hoops camp. Anything on the NBA?  

In [15]:
graph_client.search_entities('Timeline', 'nba')[:10]

,entity_name,entity_type,entity_id,entity_aliases,timeline_type,related_entities
0,NBA Playoffs Begin,Timeline,5a676e90-f3d2-4efa-ae41-1920c51fe819,[],Sports,[{'entity_id': '5523a841-e0ef-442b-b758-a01825...
1,NBA Finals Start Dates,Timeline,c48f4ffb-419b-4705-ada3-8071511ad687,[],Sports,[{'entity_id': '5523a841-e0ef-442b-b758-a01825...
2,NBA Finals End Dates,Timeline,c87f4de0-ffd4-4663-b905-2e935cf02a7c,[],Sports,[{'entity_id': '5523a841-e0ef-442b-b758-a01825...
3,NBA 2K Video Game Releases,Timeline,6325a2bf-8b20-4cce-a806-a73cff01f3be,[],Product Releases,[{'entity_id': '47cb01e2-47d7-4b15-91e2-497ee8...
4,NBA All-Star Game,Timeline,01f67971-0fe5-450e-9215-dac10e2b1366,[],Sports,[{'entity_id': '5523a841-e0ef-442b-b758-a01825...
5,National Basketball Association (NBA) Draft,Timeline,7b80679f-165f-4af5-b4ce-c84e13fb74a5,[],Sports,[{'entity_id': '5523a841-e0ef-442b-b758-a01825...
6,Novabase (PT:NBA) Earnings Releases,Timeline,c3634253-485f-430a-a869-580b17c1ce73,[],Earnings,[{'entity_id': 'e8e36842-d63f-4d89-8523-f00539...
7,Novabase Buyback Tranche Update,Timeline,b7139f4c-3cb0-4956-9b86-38baefefc601,[],Buyback Tranche Update,[{'entity_id': '25014370-ce33-4e5c-8bde-14facb...
8,Major Multi-Console Video Game Releases in Nor...,Timeline,4a5b725b-4747-4dc1-872a-68d4fefb3b69,[],Product Releases,[{'entity_id': 'ec9ae1ac-33c6-4e44-b033-00377c...
9,Novabase Corporate Guidance - New/Confirmed,Timeline,9a15a333-f22b-486c-b79c-c96f71c63e1a,[],Corporate Guidance - New/Confirmed,[{'entity_id': '4c1d559f-6b9c-4dff-b050-d2676c...


In [16]:
timeline_df1 = graph_client.search_entities('Timeline', 'Weather')
timeline_df1['entity_name']

0     Global Cold Waves (as defined by U.S. National...
1     U.S. Weather and Climate Disasters Since 2017 ...
2     U.S. Weather and Climate Disasters Since 2017 ...
3                             Postponed SpaceX Launches
4                  Dish Network Distribution Agreements
5                                 IBM Watson Milestones
6                             FedEx Service Disruptions
7     Unmanned Aerial Vehicle (Drone) Technology Mil...
8                              U.S. Retail Tax Holidays
9      Charter Communications Partnership Announcements
10              Major U.S. Oil Supply Chain Disruptions
11            Major Global Oil Supply Chain Disruptions
12                      Volkswagen Recall Announcements
13                        Dish Network Channel Removals
14                     Red Hat Contract Wins since 2010
15    Significant Precipitation During 2014-2016 Cal...
16        IBM Cloud Computing Partnership Announcements
17         Airline Crashes With Pratt & Whitney 

In [17]:
timeline_df1 = graph_client.search_entities('Timeline', 'Weather')
#timeline_df1['entity_name']
timeline_df1[timeline_df1['timeline_type'] == 'Natural Disasters']
#timeline_df1

,entity_name,entity_type,entity_id,entity_aliases,timeline_type,related_entities
1,U.S. Weather and Climate Disasters Since 2017 ...,Timeline,983f187d-ac0c-4beb-aa70-50316d29d346,[],Natural Disasters,[{'entity_id': '8fee99e4-9418-49b0-8a87-76449a...
2,U.S. Weather and Climate Disasters Since 2017 ...,Timeline,a0b14478-83fa-4e81-81b0-c2334d5cecd5,[],Natural Disasters,[{'entity_id': '8fee99e4-9418-49b0-8a87-76449a...


In [18]:
timeline_df1 = graph_client.search_entities('Timeline', 'Storm Landfalls')
#timeline_df1['entity_name']
#timeline_df1[timeline_df1['timeline_type'] == 'Natural Disasters']

for i in timeline_df1['entity_name']:
    print(i)

Atlantic Hurricane and Tropical Storm Landfalls
West Pacific Typhoon and Tropical Storm Landfalls
Atlantic Hurricane and Tropical Storm Landfalls in the United States
East Pacific Hurricane and Tropical Storm Landfalls
West Pacific Typhoon and Tropical Storm Landfalls in Japan
East Pacific Hurricane and Tropical Storm Landfalls in Mexico
Atlantic Hurricane and Tropical Storm Landfalls in Mexico
West Pacific Typhoon and Tropical Storm Landfalls in China
United Kingdom (U.K.) and European Hurricane and Tropical Storm Landfalls
Activision Blizzard, Inc. moves by 5% or more in a day
Activision Blizzard Inc (ATVI) Earnings Releases
Storm Real Estate Buyback Tranche Update
West Pacific Typhoon and Tropical Storm Landfalls in the Philippines
Activision Blizzard Inc Corporate Guidance - New/Confirmed
Activision Blizzard, Inc. Company Conference Presentations
Storm Real Estate (NO:STOR) Earnings Releases
Activision Blizzard Inc Key Personnel Role Start Dates
Activision Blizzard Inc Buyback Tran

## Get Functions

#### You mentioned nodes and edges. What kind of relationships do you have for various equities?  

In [26]:
graph_client.get_class_relationships('Equity')

,relationship_id,related_class
0,EquityConcepts,Concept
1,Peers,Equity
2,EquitySuppliers,Equity
3,EquityTimelines,Timeline
4,EquityConsumers,Equity


#### Let's take a look at Apple:

In [27]:
entity_info = graph_client.search_entities('Equity', 'AMZN')
entity_info

,entity_name,entity_type,entity_id,entity_aliases,ticker_name,sector_name,sector_id
0,"Amazon.com, Inc. (AMZN)",Equity,cfb73e32-4fa8-4701-898c-6c39279bbdb4,"[Amazon.com, Inc., Amazon company, Amazon Web ...",AMZN,Internet & Direct Marketing Retail,e382cd40-cbbe-484c-a853-1df5dd4bb089


In [28]:
entity_info['entity_id']

0    cfb73e32-4fa8-4701-898c-6c39279bbdb4
Name: entity_id, dtype: object

#### Example One: Concepts - What industries is Apple involved in? 

In [29]:
entity_concepts = graph_client.get_related_entities(entity_info['entity_id'], 'EquityConcepts')
entity_concepts

,entity_name,entity_type,entity_id,entity_aliases
0,Tablets,Concept,8a384f5c-4673-4c5e-a776-4780f4316161,[tablet computers]
1,Smart Homes,Concept,5ef8c951-f859-463a-9b69-e0cd55d1dace,"[home automation, connected home, internet of ..."
2,Digital Assistants,Concept,c8114c49-9a5a-4ab1-b152-f03e99aa5731,[virtual assistant]
3,Streaming Music,Concept,505b8e2a-bef4-4452-83c6-8635acb8462a,"[streaming music services, music streaming ser..."
4,Streaming Video,Concept,45b36735-15db-4b45-b2c6-8739f7365377,"[streaming video services, video sharing websi..."
5,Data Storage,Concept,b17035bf-de7c-4b6b-941f-1badb581b985,"[data management, cloud storage]"
6,E-Commerce Platforms,Concept,5a090fc8-38e8-4811-8827-da54cf71ed8a,[]


#### Example Two: Peers - Who are Apple's competitors? 

In [30]:
entity_peers = graph_client.get_related_entities(entity_info['entity_id'], 'Peers')
entity_peers

,entity_name,entity_type,entity_id,entity_aliases,ticker_name,sector_name,sector_id
0,Apple Inc. (AAPL),Equity,f9d09c10-c6de-4e8f-9624-996774888bcc,"[Apple Computers, Apple comp, Apple Computer, ...",AAPL,"Technology Hardware, Storage & Peripherals",2da217ab-0575-4778-9397-8e103a7b74b3
1,"Facebook, Inc. (FB)",Equity,d8ce5287-c2c4-4c5a-8ec7-334b8213de4b,"[Thefacebook.com, The Facebook, Facebook, Inc....",FB,Interactive Media & Services,ad4d6655-4996-424c-b11c-342d2e47dfa0
2,Alphabet Inc. (GOOG),Equity,37c53a09-309b-40bc-b165-90f0aac44366,"[Google, Alphabet Corp, Alphabet Inc, Abc.xyz,...",GOOG,Interactive Media & Services,ad4d6655-4996-424c-b11c-342d2e47dfa0
3,Alphabet Inc. (GOOG.L),Equity,c03414b5-bf25-4f64-b5c8-ae50fc5b9e14,"[Google, Alphabet Corp, Alphabet Inc, Abc.xyz,...",GOOG.L,Interactive Media & Services,ad4d6655-4996-424c-b11c-342d2e47dfa0
4,Microsoft Corporation (MSFT),Equity,06826ee5-f5e2-4342-9bee-1a546db3b03b,"[Mikrosoft, Windows Corporation, Microsoft.com...",MSFT,Systems Software,2dde90cc-a6b6-48df-812d-71f7854aaaec
5,"Netflix, Inc. (NFLX)",Equity,8145cd22-43cb-4911-8b91-d3fa8f69cc9d,"[Netflix Inc., Netflix.com, NetFlicks, Net Fli...",NFLX,Movies & Entertainment,9e1fd6e9-4d40-4e8a-b60a-30e971d72b63
6,Altaba Inc. (AABA),Equity,6b7f93a8-cb9b-4358-a2a8-a1152377f4fc,"[Yahoo! Inc., Yahoo Alibaba Group, Yahoo, Yahoo!]",AABA,Internet & Direct Marketing Retail,e382cd40-cbbe-484c-a853-1df5dd4bb089


#### Example Three: Suppliers - Tell me about Apple's supply chain.  

In [31]:
entity_suppliers = graph_client.get_related_entities(entity_info['entity_id'], 'EquitySuppliers')
entity_suppliers.sort_values('entity_name').head(20)

,entity_name,entity_type,entity_id,entity_aliases,ticker_name,sector_name,sector_id
0,"Atlas Air Worldwide Holdings, Inc. (AAWW)",Equity,03fd3f96-41f9-4f2d-81ca-1366c2b7ab57,[],AAWW,Air Freight & Logistics,94480747-df52-441b-8e44-8b1ee6ef31f4
3,"Escalade, Incorporated (ESCA)",Equity,0d612ace-e8ef-4b39-8097-98db44498798,[],ESCA,Leisure Products,080e5242-7408-43e0-bf0e-c5958c21f03b
1,Interroll Holding AG (INRN),Equity,7a632584-6273-4896-9cf3-9f7f19af8a9e,[],INRN,Industrial Machinery,befb8d13-cfaf-4061-9f9d-a9d59a032285
2,Nielsen Holdings plc (NLSN),Equity,5f99cc48-26cc-4d73-9f74-399871dbc965,"[A.C. Nielsen Co., The Nielsen Company, Nielse...",NLSN,Research & Consulting Services,c5a762e2-e4ee-4fbb-b57d-219ba64fe2b1
6,Technicolor SA (TCH),Equity,39deec9e-8cd8-48f2-a671-5cdf960f04e9,[],TCH,Movies & Entertainment,9e1fd6e9-4d40-4e8a-b60a-30e971d72b63
4,UNITEDLABELS Aktiengesellschaft (ULC),Equity,5a02c770-3a34-45a1-bfda-4677734c56bd,[],ULC,Distributors,126ea86a-b5e9-4705-88e5-5288775cf170
5,"United Parcel Service, Inc. (UPS)",Equity,c6423475-dff1-4015-8d84-1cee8d6bf85c,"[United Parcel Service Incorporated, United Pa...",UPS,Air Freight & Logistics,94480747-df52-441b-8e44-8b1ee6ef31f4


##### Entity Suppliers

In [32]:
entity_suppliers['sector_name'].value_counts()

Air Freight & Logistics           2
Research & Consulting Services    1
Distributors                      1
Movies & Entertainment            1
Industrial Machinery              1
Leisure Products                  1
Name: sector_name, dtype: int64

#### Example Four: Consumers - Who are the major corporate consumers of Apple products?  

In [33]:
entity_consumers = graph_client.get_related_entities(entity_info['entity_id'], 'EquityConsumers')
#entity_consumers.sort_values('entity_name').head()
entity_consumers

""


#### Example Five: Events & Timelines - What Apple events are there in the Kensho Knowledge Graph? 

In [34]:
graph_client.get_related_entities(entity_info['entity_id'], 'EquityTimelines')

,entity_name,entity_type,entity_id,entity_aliases,timeline_type,related_entities
0,Amazon (AMZN) Earnings Releases,Timeline,1ad6db0a-7c74-4795-975c-e91a2728a72c,[],Earnings,[{'entity_id': 'cfb73e32-4fa8-4701-898c-6c3927...
1,"Amazon.com, Inc. moves by 5% or more in a day",Timeline,1afb9fc5-e09a-40f6-921e-eacd5d523bec,[],Technical Indicators,[{'entity_id': 'cfb73e32-4fa8-4701-898c-6c3927...
2,Amazon Corporate Guidance - Lowered,Timeline,fab561a6-5b7a-4368-a2a0-124605693344,[],Corporate Guidance - Lowered,[{'entity_id': '96ab5874-3751-4045-b6f0-3ed4cc...
3,Amazon Corporate Guidance - Raised,Timeline,7b56fc42-7c30-4f26-a81e-9f7d1bba7413,[],Corporate Guidance - Raised,[{'entity_id': 'cfb73e32-4fa8-4701-898c-6c3927...
4,Amazon Corporate Guidance - New/Confirmed,Timeline,19beb17a-05dd-40cb-b611-3734f8da64be,[],Corporate Guidance - New/Confirmed,[{'entity_id': 'cfb73e32-4fa8-4701-898c-6c3927...
5,Amazon Shareholder/Analyst Calls,Timeline,a08f96b9-4323-4709-a50e-2c9e163866a6,[],Shareholder/Analyst Calls,[{'entity_id': '7381d04a-3dee-4c4d-b3ae-8dce95...
6,Amazon Regulatory Agency Inquiries,Timeline,85d5b78b-9ef4-4303-8364-2388138d0b47,[],Regulatory Agency Inquiries,[{'entity_id': 'cfb73e32-4fa8-4701-898c-6c3927...
7,Amazon Annual General Meeting,Timeline,b5e761ce-554d-401b-b592-bad126d5454f,[],Annual General Meeting,[{'entity_id': '3aab1ede-92b6-47f1-8deb-68acb1...
8,Amazon Discontinued Operations/Downsizings,Timeline,5ee12837-b486-4901-a9a6-1238d449452c,[],Discontinued Operations/Downsizings,[{'entity_id': 'cfb73e32-4fa8-4701-898c-6c3927...
9,Amazon Buyback Tranche Update,Timeline,7fab02de-689b-4f50-8368-2c1af1ff0c81,[],Buyback Tranche Update,[{'entity_id': '25014370-ce33-4e5c-8bde-14facb...


## More Events & Timelines 

#### What upcoming events should I know about? 

In [35]:
next_week = graph_client.get_calendar('2018-08-13', '2018-08-23')
next_week.head(20)

,event_date,event_date_precision,event_name,event_description,timeline_id,timeline_name,timeline_type,related_entities
0,2018-08-13,YYYY-MM-DD,"Reading International, Inc. Earnings Call (Q3 ...",,b1cfae56-1ee6-4de0-9e1a-f339e7d4e244,Reading International Inc (RDI) Earnings Releases,Earnings,[{'entity_id': 'cd286193-dada-4ef8-a40b-6f7709...
1,2018-08-13,YYYY-MM-DD,"Kawase Computer Supplies Co., Ltd. Earnings Re...",,0cda31a7-0b4a-4dcf-a79f-6280076b4aa9,Kawase Computer Supplies (JP:7851) Earnings Re...,Earnings,[{'entity_id': 'eb622b7c-1b54-4782-9425-02fa16...
2,2018-08-13,YYYY-MM-DD,"Electric Power Development Co., Ltd. announces...",,9e33eed2-1d4f-4b4f-83db-1e41405485f5,Independent Power and Renewable Electricity Pr...,M&A,[{'entity_id': '8383bcce-3041-4196-8679-596c2d...
3,2018-08-13,YYYY-MM-DD,Infineon Technologies and JD group announce pa...,Infineon Technologies and JD Group announce a ...,cd7688e7-22b8-444a-867b-1fb38ba4978a,Recent Key Company Developments,Corporate,[{'entity_id': '5ef8c951-f859-463a-9b69-e0cd55...
4,2018-08-13,YYYY-MM-DD,Tesla CEO Elon Musk details go-private Tesla plan,Tesla Chief Executive Officer (CEO) and Chairm...,cd7688e7-22b8-444a-867b-1fb38ba4978a,Recent Key Company Developments,Corporate,[{'entity_id': '9db61e3b-07d6-4318-85d4-370cc5...
5,2018-08-13,YYYY-MM-DD,Netflix announces CFO David Wells to step down,Netflix announces that Chief Financial Officer...,cd7688e7-22b8-444a-867b-1fb38ba4978a,Recent Key Company Developments,Corporate,[{'entity_id': '8145cd22-43cb-4911-8b91-d3fa8f...
6,2018-08-13,YYYY-MM-DD,Microsoft announces release of Azure Stack for...,Microsoft announces the release of Microsoft A...,cd7688e7-22b8-444a-867b-1fb38ba4978a,Recent Key Company Developments,Corporate,[{'entity_id': 'b17035bf-de7c-4b6b-941f-1badb5...
7,2018-08-13,YYYY-MM-DD,AMD announces launch of second generation AMD ...,AMD announces that the second generation AMD R...,cd7688e7-22b8-444a-867b-1fb38ba4978a,Recent Key Company Developments,Corporate,[{'entity_id': '778bcde1-adc4-4b6a-aa10-638d91...
8,2018-08-13,YYYY-MM-DD,Walt Disney's ESPN announces acquisition of ri...,The Walt Disney Company's ESPN announces that ...,cd7688e7-22b8-444a-867b-1fb38ba4978a,Recent Key Company Developments,Corporate,[{'entity_id': '45b36735-15db-4b45-b2c6-8739f7...
9,2018-08-13,YYYY-MM-DD,Facebook acquires Vidpresso,Vidpresso announces that its team and technolo...,cd7688e7-22b8-444a-867b-1fb38ba4978a,Recent Key Company Developments,Corporate,[{'entity_id': 'd8ce5287-c2c4-4c5a-8ec7-334b82...


#### Let's narrow that down.. What earnings-related events are coming up?  

In [36]:
next_week[next_week['event_name'].str.lower().str.contains('earning')].sort_values('event_name')[['event_date', 'event_date_precision', 'event_name']].head(10)

,event_date,event_date_precision,event_name
2509,2018-08-14 20:14:00,YYYY-MM-DDTHH:mmZ,"1347 Property Insurance Holdings, Inc. Earning..."
4473,2018-08-16 20:30:00,YYYY-MM-DDTHH:mmZ,"21Vianet Group, Inc. Earnings Release (Q2 2018)"
4481,2018-08-16 20:30:00,YYYY-MM-DDTHH:mmZ,"21Vianet Group, Inc. Reports Unaudited Consoli..."
4477,2018-08-16 20:30:00,YYYY-MM-DDTHH:mmZ,"21Vianet Group, Inc. Reports Unaudited Consoli..."
3709,2018-08-15 23:05:00,YYYY-MM-DDTHH:mmZ,3P Learning Limited Announces Audited Consolid...
3845,2018-08-16 00:00:00,YYYY-MM-DD,58.Com Inc. Announces Unaudited Consolidated E...
3708,2018-08-15 23:00:00,YYYY-MM-DDTHH:mmZ,58.com Inc. Earnings Release (Q2 2018)
3944,2018-08-16 00:29:00,YYYY-MM-DDTHH:mmZ,"8990 Holdings, Inc. Reports Earnings Results f..."
4776,2018-08-17 05:55:00,YYYY-MM-DDTHH:mmZ,A.P. Møller - Mærsk A/S Announces Consolidated...
4778,2018-08-17 05:55:00,YYYY-MM-DDTHH:mmZ,A.P. Møller - Mærsk A/S Earnings Release (Q2 2...


#### What about conferences? 

In [37]:
next_week[next_week['event_name'].str.lower().str.contains('conference')].sort_values('event_name')[['event_date', 'event_date_precision', 'event_name']].head(10)

,event_date,event_date_precision,event_name
5991,2018-08-20 00:00:00,YYYY-MM-DDTHH:mmZ,2018 Asia Petrochemical Industry Conference (2...
2999,2018-08-15 00:00:00,YYYY-MM-DD,3M Company Presents at CFA Intellisight 2018 I...
3512,2018-08-15 16:15:00,YYYY-MM-DDTHH:mmZ,"ALLETE, Inc. Presents at CFA Intellisight 2018..."
3500,2018-08-15 15:30:00,YYYY-MM-DDTHH:mmZ,"ASV Holdings, Inc. Presents at CFA Intellisigh..."
2175,2018-08-14 12:00:00,YYYY-MM-DDTHH:mmZ,"Achaogen, Inc. Presents at 2018 Wedbush PacGro..."
6547,2018-08-20 20:00:00,YYYY-MM-DDTHH:mmZ,Adobe Systems Incorporated Presents at Gartner...
6579,2018-08-20 20:45:00,YYYY-MM-DDTHH:mmZ,Advantage Oil & Gas Ltd. Presents at EnerCom’s...
3540,2018-08-15 17:55:00,YYYY-MM-DDTHH:mmZ,"Aimmune Therapeutics, Inc. Presents at 2018 We..."
6666,2018-08-21 00:00:00,YYYY-MM-DD,"Alarm.com Holdings, Inc. Presents at Raymond J..."
2220,2018-08-14 12:35:00,YYYY-MM-DDTHH:mmZ,"Albireo Pharma, Inc. Presents at 2018 Wedbush ..."


## Concepts (i.e., Thematic Search)

##### I mentioned you can search a company and see what concepts/industries/themes it is involved with. What if you want to approach this in reverse? The Kensho Knowledge Graph is recursive, meaning you can also see a list of companies involved in said a concept/industry/theme (e.g., wearables). 

In [35]:
graph_client.list_entities_of_class('Concept')

,entity_name,entity_type,entity_id,entity_aliases
0,Drones,Concept,d1d39e88-4673-493e-8362-db486006ce1e,"[unmanned aircraft systems, unmanned aerial ve..."
1,Alcohol,Concept,78218550-cdb3-494d-b5de-c257e02a82b0,[]
2,Cryptocurrency,Concept,e7ffc49f-3f96-4925-bf14-63d2438b1a8e,"[blockchain, crypto currency]"
3,Streaming Video,Concept,45b36735-15db-4b45-b2c6-8739f7365377,"[streaming video services, video sharing websi..."
4,Digital Payments,Concept,96d1bc76-938e-41a2-ac17-be05a71c98ae,"[mobile payments, e-commerce payment system, o..."
5,Wireless Charging,Concept,12552d8a-cee7-4d21-9b97-94bbc74d37d6,"[inductive charging, cordless charging]"
6,Autonomous Vehicles,Concept,257edbf7-6185-4584-a5c4-91ac0b7bcbb3,"[driverless cars, self-driving cars, robot car..."
7,E-Commerce Platforms,Concept,5a090fc8-38e8-4811-8827-da54cf71ed8a,[]
8,Cybersecurity,Concept,5fa35db7-3437-4853-9802-4a6490b7f0ce,"[computer security, it security, cybersecurity]"
9,Television,Concept,ff4d38f7-406a-4184-b6f3-498a0aeb2d1f,[]


#### What companies are involved in wearables? 

In [36]:
graph_client.get_related_entities('1f0667ed-3009-463a-9301-bfe158f863a8', 'ConceptEquities')

,entity_name,entity_type,entity_id,entity_aliases,ticker_name,sector_name,sector_id
0,Apple Inc. (AAPL),Equity,f9d09c10-c6de-4e8f-9624-996774888bcc,"[Apple Computers, Apple comp, Apple Computer, ...",AAPL,"Technology Hardware, Storage & Peripherals",2da217ab-0575-4778-9397-8e103a7b74b3
1,Garmin Ltd. (GRMN),Equity,67958864-0c6c-4fef-851f-ff43fd196212,"[Garmin International, Inc., Garmin Internatio...",GRMN,Consumer Electronics,fc9bd158-ebba-433a-88ce-ed389cbe1c84
2,"Skyworks Solutions, Inc. (SWKS)",Equity,941ef5a5-19cd-436d-a8df-18af60742539,"[Skyworks Solutions Inc., Skyworks Solutions I...",SWKS,Semiconductors,c1631e7e-1ea3-4225-bc28-3b7dd8a9ae1b
3,Amphenol Corporation (APH),Equity,e9b43a37-9718-4f2b-bee4-6a7747e28f7f,[],APH,Electronic Components,19a722a6-e905-4656-945c-1d5022b06757
4,Michael Kors Holdings Limited (KORS),Equity,18a3abf4-6b89-4c09-b1d0-df541b4f4b8b,[Michael Kors Holdings],KORS,"Apparel, Accessories & Luxury Goods",56693b1f-f831-4de8-9e1a-da144ea29053
5,QUALCOMM Incorporated (QCOM),Equity,62ad3190-eb46-4824-bbe0-5e1ca3657fea,"[QUALCOMM, Qualcomm Incorporated]",QCOM,Semiconductors,c1631e7e-1ea3-4225-bc28-3b7dd8a9ae1b
6,Western Digital Corporation (WDC),Equity,ee1bda7a-af07-4be8-89de-a46335b7934e,[Western Digital],WDC,"Technology Hardware, Storage & Peripherals",2da217ab-0575-4778-9397-8e103a7b74b3
7,"Analog Devices, Inc. (ADI)",Equity,39182bbd-5bf6-4bc2-82df-99b28521229a,"[Analog Devices, Inc., Analog Devices Inc, Ana...",ADI,Semiconductors,c1631e7e-1ea3-4225-bc28-3b7dd8a9ae1b
8,Broadcom Inc. (AVGO),Equity,130b60d9-7078-4079-ba70-dc4b818aeaec,[],AVGO,Semiconductors,c1631e7e-1ea3-4225-bc28-3b7dd8a9ae1b
9,"Qorvo, Inc. (QRVO)",Equity,0ebbdfd6-d8e0-4a55-b0fb-b66a1498832c,[Qorvo],QRVO,Semiconductors,c1631e7e-1ea3-4225-bc28-3b7dd8a9ae1b


In [13]:
print("Equity Info")
entity_info = graph_client.search_entities('Equity', 'MSFT')
entity_info

Equity Info


,entity_name,entity_type,entity_id,entity_aliases,ticker_name,sector_name,sector_id
0,Microsoft Corporation (MSFT),Equity,06826ee5-f5e2-4342-9bee-1a546db3b03b,"[Mikrosoft, Windows Corporation, Microsoft.com...",MSFT,Systems Software,2dde90cc-a6b6-48df-812d-71f7854aaaec


In [14]:
print("Equity Concepts")
entity_concepts = graph_client.get_related_entities(entity_info['entity_id'], 'EquityConcepts')
entity_concepts

Equity Concepts


,entity_name,entity_type,entity_id,entity_aliases
0,Video Gaming,Concept,56da58ef-10f5-4333-a7b0-f06c3fedbd33,"[electronic gaming, computer games, video games]"
1,Tablets,Concept,8a384f5c-4673-4c5e-a776-4780f4316161,[tablet computers]
2,Data Storage,Concept,b17035bf-de7c-4b6b-941f-1badb581b985,"[data management, cloud storage]"
3,Digital Assistants,Concept,c8114c49-9a5a-4ab1-b152-f03e99aa5731,[virtual assistant]
4,Cybersecurity,Concept,5fa35db7-3437-4853-9802-4a6490b7f0ce,"[computer security, it security, cybersecurity]"
5,Virtual Reality,Concept,2eaea590-2764-49ea-b64c-39c340e3b4f9,[vr]
6,Smart Homes,Concept,5ef8c951-f859-463a-9b69-e0cd55d1dace,"[home automation, connected home, internet of ..."
7,Smartphones,Concept,fa23a8bc-07e2-4bf2-970c-de68b7c3f431,"[smartphone, smart phone, smart phones]"
8,Social Networks,Concept,2805c9c2-8a5e-44ae-b7b7-7e61aa9d6e89,"[social networking site, social networking ser..."
9,Streaming Music,Concept,505b8e2a-bef4-4452-83c6-8635acb8462a,"[streaming music services, music streaming ser..."


In [15]:
print("Peers")
entity_peers = graph_client.get_related_entities(entity_info['entity_id'], 'Peers')
entity_peers

Peers


,entity_name,entity_type,entity_id,entity_aliases,ticker_name,sector_name,sector_id
0,Apple Inc. (AAPL),Equity,f9d09c10-c6de-4e8f-9624-996774888bcc,"[Apple Computers, Apple comp, Apple Computer, ...",AAPL,"Technology Hardware, Storage & Peripherals",2da217ab-0575-4778-9397-8e103a7b74b3
1,"Amazon.com, Inc. (AMZN)",Equity,cfb73e32-4fa8-4701-898c-6c39279bbdb4,"[Amazon.com, Inc., Amazon company, Amazon Web ...",AMZN,Internet & Direct Marketing Retail,e382cd40-cbbe-484c-a853-1df5dd4bb089
2,"salesforce.com, inc. (CRM)",Equity,5360a8c7-35c0-40ff-b819-20a9bfa83326,"[SalesForce, Salesforce.com]",CRM,Application Software,d173532e-b125-401d-87c1-c4076e748c69
3,"Facebook, Inc. (FB)",Equity,d8ce5287-c2c4-4c5a-8ec7-334b8213de4b,"[Thefacebook.com, The Facebook, Facebook, Inc....",FB,Interactive Media & Services,ad4d6655-4996-424c-b11c-342d2e47dfa0
4,Alphabet Inc. (GOOG),Equity,37c53a09-309b-40bc-b165-90f0aac44366,"[Google, Alphabet Corp, Alphabet Inc, Abc.xyz,...",GOOG,Interactive Media & Services,ad4d6655-4996-424c-b11c-342d2e47dfa0
5,Alphabet Inc. (GOOG.L),Equity,c03414b5-bf25-4f64-b5c8-ae50fc5b9e14,"[Google, Alphabet Corp, Alphabet Inc, Abc.xyz,...",GOOG.L,Interactive Media & Services,ad4d6655-4996-424c-b11c-342d2e47dfa0
6,Altaba Inc. (AABA),Equity,6b7f93a8-cb9b-4358-a2a8-a1152377f4fc,"[Yahoo! Inc., Yahoo Alibaba Group, Yahoo, Yahoo!]",AABA,Internet & Direct Marketing Retail,e382cd40-cbbe-484c-a853-1df5dd4bb089
7,Oracle Corporation (ORCL),Equity,b947ea63-eac3-4375-abee-333b1db74b85,"[Oracle Corp., Oracle Corp, Oracle Media Server]",ORCL,Systems Software,2dde90cc-a6b6-48df-812d-71f7854aaaec
8,"VMware, Inc. (VMW)",Equity,5569f0a7-3703-4506-8e13-48da4fb0eaa4,[],VMW,Systems Software,2dde90cc-a6b6-48df-812d-71f7854aaaec
9,The Sage Group plc (SGE),Equity,d8ca480b-49b0-44ef-a7c4-c7a4812b0a38,[],SGE,Application Software,d173532e-b125-401d-87c1-c4076e748c69


In [160]:
print("Suppliers")
entity_suppliers = graph_client.get_related_entities(entity_info['entity_id'], 'EquitySuppliers')
entity_suppliers.sort_values('entity_name')

Suppliers


,entity_name,entity_type,entity_id,entity_aliases,ticker_name,sector_name,sector_id
25,B2BPartner S.A. (B2B),Equity,b7b634d9-a090-4b35-af4a-aad7278e5b2f,[],B2B,Internet & Direct Marketing Retail,e382cd40-cbbe-484c-a853-1df5dd4bb089
1,CHEMOMETEC A/S (CHEMM),Equity,582dc6e7-bfcf-432c-8e23-36da7c1c2285,[],CHEMM,Life Sciences Tools & Services,e5a4cea6-1a4a-4149-b1dd-bc9ef9f8614c
4,CPMC Holdings Limited (0906),Equity,2f6d4e74-a52d-4fa0-9b6f-2b4e1611312f,[],0906,Metal & Glass Containers,56ef799a-f26f-4ede-ba42-966b4be1f2f6
6,"Catalent, Inc. (CTLT)",Equity,b55e17e1-8ce2-40fe-a8ab-c0007159095d,[],CTLT,Pharmaceuticals,31d8996f-85d3-4920-9c15-83b3ab49b152
13,Cello Health plc (CLL),Equity,fd14f573-6d3d-4734-9c77-7738ebd5943e,[],CLL,Advertising,f83e6b06-b017-4e63-b9dc-9cd5eebf37b2
24,Coheris SA (COH),Equity,70ca2684-2f3d-4eec-89bc-cf6018f6351b,[],COH,Application Software,d173532e-b125-401d-87c1-c4076e748c69
16,"Edge Therapeutics, Inc. (EDGE)",Equity,8d8a9c8d-0e95-4956-9757-4919be27da79,[],EDGE,Biotechnology,9993c9c3-afef-4a66-acc2-1a54694eb9d3
28,Flex Ltd. (FLEX),Equity,395c2cf5-6764-4d76-b65c-64f519932283,[],FLEX,Electronic Manufacturing Services,9cba514c-b902-4623-9e20-f8402dad25f1
14,"Foundation Medicine, Inc. (FMI)",Equity,7d135081-ddbc-4873-857f-35be9ca70254,[],FMI,Biotechnology,9993c9c3-afef-4a66-acc2-1a54694eb9d3
7,"Guangzhou Boji Medical & Biotechnological Co.,...",Equity,0bb287b6-fb5a-4c00-b867-eeb1c604106e,[],300404,Life Sciences Tools & Services,e5a4cea6-1a4a-4149-b1dd-bc9ef9f8614c


In [155]:
entity_suppliers['sector_name'].value_counts()

Electronic Equipment & Instruments    1
Electronic Manufacturing Services     1
Application Software                  1
Name: sector_name, dtype: int64

In [161]:
print("Consumers")
entity_consumers = graph_client.get_related_entities(entity_info['entity_id'], 'EquityConsumers')
#entity_consumers.sort_values('entity_name').head()
entity_consumers

Consumers


""
